# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv("cities.csv")
city_weather


FileNotFoundError: [Errno 2] No such file or directory: '..//starter_code/cities.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key=g_key)

locations = city_weather[["lat", "lon"]]

humidity = city_weather["humidity"]

In [26]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
best_weather_df = city_weather.loc[(city_weather["speed"] <=10) & (city_weather["all"] ==0) &  
                                   (city_weather["temp_max"] >= 70) & (city_weather["temp_max"] <=80)].dropna()

best_weather_df

,Unnamed: 0,city,lat,lon,temp_max,humidity,all,speed,country,date
40,40,Samandag,36.0842,35.9771,77.36,38,0,5.39,TR,[]
42,42,Tura,25.5198,90.2201,78.67,65,0,4.00,IN,[]
107,107,Cabo San Lucas,22.8909,-109.9124,71.51,66,0,6.64,MX,[]
289,289,Thabazimbi,-24.5917,27.4116,79.21,35,0,3.76,ZA,[]
378,378,North Myrtle Beach,33.8160,-78.6800,79.11,76,0,9.22,US,[]
393,393,Coahuayana,18.7333,-103.6833,75.42,49,0,2.17,MX,[]
424,424,Cap Malheureux,-19.9842,57.6142,78.48,72,0,5.01,MU,[]
427,427,Erzin,36.9559,36.2000,75.70,38,0,4.88,TR,[]
502,502,Beyneu,45.3167,55.2000,74.16,24,0,9.66,KZ,[]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = best_weather_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ''

hotel_df.head()

,Unnamed: 0,city,lat,lon,temp_max,humidity,all,speed,country,date,Hotel Name
0,40,Samandag,36.0842,35.9771,77.36,38,0,5.39,TR,[],
1,42,Tura,25.5198,90.2201,78.67,65,0,4.00,IN,[],
2,107,Cabo San Lucas,22.8909,-109.9124,71.51,66,0,6.64,MX,[],
3,289,Thabazimbi,-24.5917,27.4116,79.21,35,0,3.76,ZA,[],
4,378,North Myrtle Beach,33.8160,-78.6800,79.11,76,0,9.22,US,[],


In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
             "keyword" : "hotel",
             "radius" : "5000",
             "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lon = row["lon"]
    city_name = row["city"]
    
    params["location"] = f"{lat},{lon}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("---------------------------------")

Retrieving Results for Index 0: Samandag.
Closest hotel in Samandag is Truva Life Hotel.
---------------------------------
Retrieving Results for Index 1: Tura.
Closest hotel in Tura is Hotel Polo Orchid.
---------------------------------
Retrieving Results for Index 2: Cabo San Lucas.
Closest hotel in Cabo San Lucas is The Cape, A Thompson Hotel.
---------------------------------
Retrieving Results for Index 3: Thabazimbi.
Closest hotel in Thabazimbi is Hardekool Guesthouse With a View.
---------------------------------
Retrieving Results for Index 4: North Myrtle Beach.
Closest hotel in North Myrtle Beach is Hampton Inn North Myrtle Beach-Harbourgate.
---------------------------------
Retrieving Results for Index 5: Coahuayana.
Closest hotel in Coahuayana is Hotel Los Arcos.
---------------------------------
Retrieving Results for Index 6: Cap Malheureux.
Closest hotel in Cap Malheureux is Zilwa Attitude Hotel.
---------------------------------
Retrieving Results for Index 7: Erzin.


In [17]:
hotel_df.head()

,Unnamed: 0,city,lat,lon,temp_max,humidity,all,speed,country,date,Hotel Name
0,40,Samandag,36.0842,35.9771,77.36,38,0,5.39,TR,[],Truva Life Hotel
1,42,Tura,25.5198,90.2201,78.67,65,0,4.00,IN,[],Hotel Polo Orchid
2,107,Cabo San Lucas,22.8909,-109.9124,71.51,66,0,6.64,MX,[],"The Cape, A Thompson Hotel"
3,289,Thabazimbi,-24.5917,27.4116,79.21,35,0,3.76,ZA,[],Hardekool Guesthouse With a View
4,378,North Myrtle Beach,33.8160,-78.6800,79.11,76,0,9.22,US,[],Hampton Inn North Myrtle Beach-Harbourgate


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))